In [20]:
import pymc3 as pm
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import theano
output_notebook()
from random import sample
from sklearn.metrics import classification_report
import pickle

Loading BokehJS ...

In [2]:
Data = pd.read_csv("points.txt")
Data = Data.drop(['point_source_ID', 'edge_of_flight_line_flag', 'user_data'], axis = 1)
Data.head(3)

,x,y,z,classification,gpstime,scan_angle,intensity,number_of_returns,return_number,direction_of_scan_flag,red_channel,green_channel,blue_channel
0,6357435.38,1920674.90,160.50,1,6.547720e+07,10,151,1,1,1,7680,8704,7424
1,6357435.38,1920675.95,160.01,2,6.547720e+07,10,78,1,1,0,7424,8960,7680
2,6357432.62,1920675.03,159.91,7,6.547720e+07,10,71,1,1,0,7424,8704,7936


In [3]:
Data['classification'].unique()

array([1, 2, 7, 5, 6])

In [4]:
Counts = Data['classification'].value_counts()
print (Counts)

5    40252
2    28543
1    22793
6    12812
7     4255
Name: classification, dtype: int64


In [5]:
Data['r'] = np.sqrt(Data['x']**2 + Data['y']**2)
Data = Data.drop(['x', 'y'], axis = 1)

In [6]:
DataCorrelations = Data.corr()
print (DataCorrelations)

                               z  classification   gpstime  scan_angle  \
z                       1.000000        0.521013 -0.017554    0.024894   
classification          0.521013        1.000000 -0.050794    0.046242   
gpstime                -0.017554       -0.050794  1.000000   -0.887780   
scan_angle              0.024894        0.046242 -0.887780    1.000000   
intensity              -0.191177       -0.332200 -0.010847    0.038204   
number_of_returns       0.328410        0.250299  0.020555   -0.030634   
return_number          -0.041576        0.053497  0.013867   -0.021743   
direction_of_scan_flag  0.001273        0.004487 -0.039010    0.042865   
red_channel            -0.054378        0.039821  0.020954   -0.044339   
green_channel          -0.027891        0.058669  0.015206   -0.037843   
blue_channel           -0.126787        0.008045  0.009596   -0.026775   
r                      -0.132905       -0.094354  0.219605   -0.278599   

                        intensity  nu

Classification is correlated with r weakly and z, intensity, number_of_returns strongly

In [7]:
RefinedData = Data.drop(['gpstime', 'scan_angle', 'direction_of_scan_flag', 'red_channel', 'green_channel', 'blue_channel'], axis = 1)
RefinedData['z'] = (RefinedData['z'] - RefinedData['z'].min()) / (RefinedData['z'].max() - RefinedData['z'].min())
RefinedData['r'] = (RefinedData['r'] - RefinedData['r'].min()) / (RefinedData['r'].max() - RefinedData['r'].min())
RefinedData['intensity'] = (RefinedData['intensity'] - RefinedData['intensity'].min()) / (RefinedData['intensity'].max() - RefinedData['intensity'].min())
RefinedData.head(3)
Data = RefinedData

In [8]:
Class1 = Data.loc[Data['classification'] == 1]
Class1Sample = Class1.sample(n =4255)
Class2 = Data.loc[Data['classification'] == 2]
Class2Sample = Class2.sample(n =4255)
Class5 = Data.loc[Data['classification'] == 5]
Class5Sample = Class5.sample(n =4255)
Class6 = Data.loc[Data['classification'] == 6]
Class6Sample = Class6.sample(n =4255)
Class7 = Data.loc[Data['classification'] == 7]
Class7Sample = Class7.sample(n =4255)

W = Class1Sample.append(Class2Sample, ignore_index=True)
W = W.append(Class5Sample, ignore_index=True)
W = W.append(Class6Sample, ignore_index=True)
WellMixedData = W.append(Class7Sample, ignore_index=True)
WellMixedData = WellMixedData.sample(frac=1)
print (WellMixedData.shape)

(21275, 6)


In [9]:
x = WellMixedData[['r', 'z', 'intensity', 'number_of_returns']].values
y = pd.get_dummies(WellMixedData['classification']).values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
print (x_train.shape)
print (y_train.shape)
NumberOfFeatures = x_train.shape[1]
NumberOfClasses = y_train.shape[1]

(17020, 4)
(17020, 5)


In [11]:
def logit(C):
    return 1./(1.+pm.math.exp(-C))

In [12]:
def CreateModel(X, Y):
    with pm.Model() as Model:
        Slopes = pm.Normal("Slopes", sd = 100., shape=(NumberOfFeatures, NumberOfClasses))
        Intercept = pm.Normal("Intercepts", sd = 100., shape=NumberOfClasses)
        LogOdds = pm.Deterministic("p", logit(theano.tensor.dot(X, Slopes) + Intercept))
        
        Observed = pm.Multinomial("Multi", p = LogOdds, observed = Y, n = 1)
        
        
    return Model

In [13]:
X = theano.shared(x_train)
Y = theano.shared(y_train)
Model = CreateModel(X, Y)

/anaconda3/lib/python3.6/site-packages/theano/tensor/basic.py:2184: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [16]:
try:
    approx = pickle.load(open("Trace.p", "rb"))
except (OSError, IOError) as e:
    
    with Model:
        inference = pm.ADVI()
        approx = pm.fit(n=100000, method=inference)
    pickle.dump(approx, open("Trace.p", "wb"))



Average Loss = 14,170: 100%|██████████| 100000/100000 [25:48<00:00, 64.59it/s]
Finished [100%]: Average Loss = 14,170


In [17]:
X.set_value(x_test)
Y.set_value(y_test)

In [57]:
Intercepts = sample_array['Intercepts']
Slopes = sample_array['Slopes']

In [58]:
Predictions = theano.tensor.dot(x_test,Slopes)+Intercepts

In [59]:
Probabilities = logit(Predictions).eval()

In [60]:
ppc_preds = (Probabilities == np.max(Probabilities, axis=1, keepdims=True))
print(classification_report(y_test, ppc_preds))

             precision    recall  f1-score   support

          0       0.87      0.11      0.20       863
          1       0.42      0.84      0.56       837
          2       0.96      0.89      0.92       857
          3       0.84      0.94      0.89       855
          4       0.66      0.55      0.60       843

avg / total       0.75      0.67      0.64      4255



In [50]:
TrainingProbabilities = logit(theano.tensor.dot(x_train,Slopes)+Intercepts).eval()

In [51]:
Training_preds = (TrainingProbabilities == np.max(TrainingProbabilities, axis=1, keepdims=True))
print(classification_report(y_train, Training_preds))

             precision    recall  f1-score   support

          0       0.85      0.13      0.23      3392
          1       0.43      0.84      0.57      3418
          2       0.95      0.89      0.92      3398
          3       0.84      0.94      0.88      3400
          4       0.66      0.54      0.59      3412

avg / total       0.74      0.67      0.64     17020



No Overfitting

In [ ]:
X.set_value(x_train)
Y.set_value(y_train)
with Model:
    trace = pm.sample(1000, cores = 4, njobs = 2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Intercepts, Slopes]
 21%|██▏       | 319/1500 [35:31<2:11:29,  6.68s/it]